This notebook applies h2o auto ML techniques to predict Redshift through linear regression models. Applying on only supernovae type IA.

In [1]:
%load_ext autoreload
%autoreload 2
import os 
import sys
import time
import h2o

import pandas as pd

from sklearn.metrics import mean_squared_error

In [2]:
module_path = os.path.abspath(os.path.join('..','..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.modeling.functions_autoML import auto_ML, save_aml_models, gen_predictions

In [3]:
pd.set_option("display.max_rows", 6)

In [4]:
df = pd.read_pickle("../../data/structured/df.pkl")
#IA Filter
df = df.loc[df.type_bool == True]
df

,desg,desi,desr,desz,Xaxis,desg_GP,desi_GP,desr_GP,desz_GP,wavelets,...,f14,f15,f16,f17,f18,f19,f20,type,type_bool,REDSHIFT_SPEC
ID,,,,,,,,,,,,,,,,,,,,,
SN10005,"[[10.027000000001863, 4.996, 8.081], [33.94500...","[[0.015000000006693881, 0.2977, 2.206], [18.94...","[[0.0, 1.033, 1.325], [13.015000000006694, 0.4...","[[0.03500000000349246, 1.826, 3.029], [48.875,...","[0.0, 1.0381919191919462, 2.0763838383838924, ...","([10.027000000001863, 10.782565656567513, 11.5...","([0.015000000006693881, 1.0530404040470573, 2....","([0.0, 1.0281717171717606, 2.056343434343521, ...","([0.03500000000349246, 1.0222929292964054, 2.0...","[2.000004411755352, 2.000003547336676, 2.00000...",...,-26.554909,15.768490,16.436792,17.114796,-20.292959,-24.222562,-11.295566,Ia,True,1.08327
SN10007,"[[0.0, 6.266, 10.48], [10.105999999999767, 17....","[[0.03200000000651926, 7.161, 9.313], [2.01600...","[[2.0, 1.24, 2.429], [10.121000000006461, 20.0...","[[0.9770000000062282, 0.6586, 5.329], [15.9930...","[0.0, 1.5136565656566285, 3.027313131313257, 4...","([0.0, 1.5132626262626456, 3.0265252525252913,...","([0.03200000000651926, 1.5451717171782005, 3.0...","([2.0, 3.49314141414145, 4.9862828282829, 6.47...","([0.9770000000062282, 2.480787878794107, 3.984...","[11.341091517194192, 8.812204685737349, 6.9798...",...,-99.877031,21.670852,24.155365,-26.622108,-27.616139,-19.981458,49.299481,Ia,True,0.82331
SN100361,"[[6.957000000002154, 16.02, 23.92], [7.9690000...","[[0.023000000001047738, 4.903, 3.324], [6.9840...","[[0.0, 0.7103, 2.626], [5.9760000000023865, 5....","[[8.055000000000291, 2.567, 2.665], [15.035000...","[0.0, 1.3031111111111275, 2.606222222222255, 3...","([6.957000000002154, 8.15910101010318, 9.36120...","([0.023000000001047738, 1.315494949496019, 2.6...","([0.0, 1.3026767676768058, 2.6053535353536117,...","([8.055000000000291, 9.27674747474778, 10.4984...","[8.233549869515556, 9.464488874691614, 10.4511...",...,102.074783,-95.727930,4.861556,-29.749071,23.442475,5.441218,12.503554,Ia,True,0.90012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SN99438,"[[0.0, 2.33, 2.55], [7.894000000000233, 3.805,...","[[0.030999999995401595, 0.006543, 3.683], [7.9...","[[0.014999999999417923, 0.4446, 1.3], [14.0039...","[[0.06199999999807915, 1.497, 4.486], [7.97199...","[0.0, 1.7255757575757058, 3.4511515151514116, ...","([0.0, 1.6948383838383538, 3.3896767676767077,...","([0.030999999995401595, 1.75597979797521, 3.48...","([0.014999999999417923, 1.7298888888882826, 3....","([0.06199999999807915, 1.7869494949475417, 3.5...","[6.789900493579756, 7.110462975693107, 7.57937...",...,-3.768615,-34.417710,32.878025,-33.942007,34.613412,61.324876,-51.210969,Ia,True,0.75966
SN99457,"[[0.0, 10.06, 4.768], [4.075000000004366, 4.39...","[[3.0429999999978463, 0.9564, 1.984], [4.10599...","[[3.004000000000815, 1.456, 1.709], [4.0900000...","[[4.137000000002445, 9.726, 4.49], [12.0469999...","[0.0, 1.7667373737373662, 3.5334747474747323, ...","([0.0, 1.7360000000000142, 3.4720000000000284,...","([3.0429999999978463, 4.778717171715048, 6.514...","([3.004000000000815, 4.729858585859406, 6.4557...","([4.137000000002445, 5.861949494951907, 7.5868...","[8.252504299669107, 11.125114322870362, 13.374...",...,-53.090074,9.078515,-15.904951,34.732243,-14.319036,-22.934570,-9.153751,Ia,True,0.80886
SN99946,"[[11.961000000002969, 1.526, 5.683], [13.05500...","[[0.01599999999598367, 4.95, 3.147], [3.034999...","[[0.0, 0.4826, 2.129], [2.995999999999185, 0.7...","[[0.03899999999703141, 1.045, 2.483], [12.0389...","[0.0, 1.3626161616161374, 2.725232323232275, 4...","([11.961000000002969, 13.202242424245336, 14.4...","([0.01599999999598367, 1.3782121212081455, 2.7...","([0.0, 1.362181818181816, 2.724363636363632, 4...","([0.03899999999703141, 1.4012222222192594, 2.7...","[3.7693816011787566, 5.06053615747151, 6.78000...",...,95.149318,183.911735,62.540835,28.895779,-271.089840,54.766397,39.959973,Ia,True,0.40625


In [10]:
df_smaller_half = df.sort_values("REDSHIFT_SPEC")[:int(len(df)/2)]
df_higher_half = df.sort_values("REDSHIFT_SPEC")[int(len(df)/2):]

df_smaller_half

,desg,desi,desr,desz,Xaxis,desg_GP,desi_GP,desr_GP,desz_GP,wavelets,...,f14,f15,f16,f17,f18,f19,f20,type,type_bool,REDSHIFT_SPEC
ID,,,,,,,,,,,,,,,,,,,,,
SN20557,"[[0.0, 4870.0, 121.0], [7.972000000001572, 148...","[[1.0, 5986.0, 122.4], [7.995999999999185, 880...","[[0.9879999999975553, 7148.0, 134.3], [7.98000...","[[5.109000000004016, 8564.0, 159.3], [8.010999...","[0.0, 0.769686868686874, 1.539373737373748, 2....","([0.0, 0.7578888888889159, 1.5157777777778318,...","([1.0, 1.7480606060605783, 2.4961212121211567,...","([0.9879999999975553, 1.7360707070682735, 2.48...","([5.109000000004016, 5.827080808084789, 6.5451...","[4057.7695564773107, 7593.933715675284, 11327....",...,1006.336537,1951.536814,1090.093779,2067.072917,-125.128986,-1530.325323,-1265.072084,Ia,True,0.04545
SN319694,"[[0.0, 0.09072, 4.786], [9.984000000004016, 0....","[[0.031000000002677552, 6.197, 5.437], [12.004...","[[2.139999999999418, 2.821, 3.03], [11.9880000...","[[12.031000000002678, 0.8086, 4.692], [23.0080...","[0.0, 1.1313535353535435, 2.262707070707087, 3...","([0.0, 1.1299292929293416, 2.2598585858586833,...","([0.031000000002677552, 1.1617979798006626, 2....","([2.139999999999418, 3.248393939393355, 4.3567...","([12.031000000002678, 13.040828282830942, 14.0...","[523.0034351208778, 289.33281794531683, 105.61...",...,7033.847950,-2564.903176,-2207.641958,2385.074475,-2563.255006,-4263.644688,-4182.809857,Ia,True,0.06905
SN46940,"[[0.0, 1285.0, 24.54], [2.997000000003027, 916...","[[0.023999999997613486, 1446.0, 27.11], [3.042...","[[0.008000000001629815, 2148.0, 40.02], [3.004...","[[0.046999999998661224, 1170.0, 21.9], [3.0750...","[0.0, 0.5949393939393698, 1.1898787878787396, ...","([0.0, 0.5944242424242236, 1.1888484848484473,...","([0.023999999997613486, 0.6184141414117708, 1....","([0.008000000001629815, 0.6024242424258535, 1....","([0.046999999998661224, 0.6414646464632972, 1....","[1287.464349921601, 1775.6575687702784, 2110.0...",...,775.972692,608.426535,-1727.714614,729.726920,-325.747033,1715.963813,-1520.018349,Ia,True,0.06939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SN823706,"[[16.932999999997264, 1.878, 3.771], [29.96800...","[[0.014999999999417923, 18.77, 5.45], [2.13999...","[[0.0, 8.31, 5.338], [2.1239999999961583, 6.20...","[[0.04599999999481952, 17.5, 4.704], [2.170999...","[0.0, 0.9090101010100845, 1.818020202020169, 2...","([16.932999999997264, 17.45939393939118, 17.98...","([0.014999999999417923, 0.923616161615593, 1.8...","([0.0, 0.8489090909091038, 1.6978181818182077,...","([0.04599999999481952, 0.9545454545403099, 1.8...","[4.881411488546439, 4.257521172445462, 3.80378...",...,-23.545191,39.697763,46.252411,8.897905,29.213838,-1.729084,6.604949,Ia,True,0.75025
SN824611,"[[6.120999999999185, 1.888, 4.101], [14.125, 6...","[[3.0200000000040745, 3.102, 2.988], [5.914000...","[[4.969000000004598, 2.396, 6.048], [13.141000...","[[0.0, 5.114, 4.046], [19.148999999997613, 0.7...","[0.0, 1.3619393939394158, 2.7238787878788315, ...","([6.120999999999185, 7.360666666665841, 8.6003...","([3.0200000000040745, 4.3414545454586255, 5.66...","([4.969000000004598, 6.270686868691451, 7.5723...","([0.0, 1.3619393939394158, 2.7238787878788315,...","[13.620995007564469, 15.649681852658569, 17.28...",...,32.370815,-33.345570,57.366604,-69.986416,-48.043029,21.371989,-19.256028,Ia,True,0.75044
SN651390,"[[0.0, 1.77, 4.765], [4.075000000004366, 3.897...","[[0.023999999997613486, 29.29, 3.197], [3.0429...","[[0.008000000001629815, 15.35, 2.153], [3.0040...","[[0.046999999998661224, 24.78, 2.528], [3.0750...","[0.0, 1.1713636363636746, 2.342727272727349, 3...","([0.0, 1.1712121212121653, 2.3424242424243307,...","([0.023999999997613486, 1.1658484848461501, 2....","([0.008000000001629815, 1.1792828282844798, 2....","([0.046999999998661224, 1.2166666666653878, 2....","[10.794658852386924, 10.39512385108209, 10.190...",...,-50.519480,7.155188,56.324397,15.267763,-1.555149,4.212241,21.389891,Ia,True,0.750

In [11]:
df_higher_half

,desg,desi,desr,desz,Xaxis,desg_GP,desi_GP,desr_GP,desz_GP,wavelets,...,f14,f15,f16,f17,f18,f19,f20,type,type_bool,REDSHIFT_SPEC
ID,,,,,,,,,,,,,,,,,,,,,
SN265619,"[[9.875, 2.511, 4.947], [12.75800000000163, 0....","[[9.695000000006985, 14.69, 9.889], [12.785000...","[[0.0, 0.3188, 1.335], [9.887000000002445, 1.2...","[[9.726000000002387, 2.628, 3.968], [12.816000...","[0.0, 1.0379595959596124, 2.075919191919225, 3...","([9.875, 10.80255555555557, 11.730111111111139...","([9.695000000006985, 10.634747474754422, 11.57...","([0.0, 1.0375252525252907, 2.0750505050505814,...","([9.726000000002387, 10.665717171719551, 11.60...","[5.758735456469932, 5.283580775828241, 4.92042...",...,125.716766,-16.985000,-21.082779,38.876036,74.859161,-46.863697,30.383455,Ia,True,0.75089
SN532981,"[[7.99199999999837, 3.011, 1.596], [13.8899999...","[[20.984000000004016, 1.035, 5.178], [22.0, 1....","[[0.0, 2.797, 1.604], [8.00800000000163, 0.800...","[[15.039000000004307, 5.616, 4.836], [23.04299...","[0.0, 1.452767676767682, 2.905535353535364, 4....","([7.99199999999837, 9.353939393937786, 10.7158...","([20.984000000004016, 22.22480808081206, 23.46...","([0.0, 1.4427474747474964, 2.885494949494993, ...","([15.039000000004307, 16.289313131317392, 17.5...","[2.510293265507222, 4.0108841846321885, 7.7641...",...,-34.741254,42.690565,-241.966264,-156.652371,-47.759781,-66.782714,-15.923177,Ia,True,0.75095
SN840017,"[[0.0, 11.28, 7.732], [8.972000000001572, 17.2...","[[0.011000000005878974, 8.117, 16.98], [9.0030...","[[5.972000000001572, 2.994, 5.568], [8.0070000...","[[7.027000000001863, 10.81, 8.901], [8.0230000...","[0.0, 1.169424242424253, 2.338848484848506, 3....","([0.0, 1.169424242424253, 2.338848484848506, 3...","([0.011000000005878974, 1.1602323232381782, 2....","([5.972000000001572, 6.969949494951053, 7.9678...","([7.027000000001863, 8.115343434345288, 9.2036...","[19.94402543977181, 23.805260322870943, 27.130...",...,-67.683121,-182.802469,-17.863691,-12.070922,-34.562926,-39.540620,52.320937,Ia,True,0.75095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SN443231,"[[2.989000000001397, 3.23, 2.585], [11.9610000...","[[0.01599999999598367, 3.493, 3.146], [12.0120...","[[0.0, 0.3684, 2.129], [2.995999999999185, 1.1...","[[0.03899999999703141, 0.6598, 2.483], [13.112...","[0.0, 1.7666565656565416, 3.533313131313083, 5...","([2.989000000001397, 4.7249090909104465, 6.460...","([0.01599999999598367, 1.7822121212081372, 3.5...","([0.0, 1.7360707070706989, 3.4721414141413978,...","([0.03899999999703141, 1.8052626262596636, 3.5...","[9.458158839154713, 7.736430467320005, 6.40274...",...,27.945910,-62.135187,29.111151,-30.903974,6.519739,46.042443,-34.203921,Ia,True,1.10919
SN694667,"[[0.0, 3.11, 4.764], [2.997000000003027, 1.525...","[[0.023999999997613486, 1.246, 3.145], [4.1059...","[[0.008000000001629815, 2.527, 2.13], [3.00400...","[[3.0750000000043656, 1.188, 3.729], [4.137000...","[0.0, 1.7667373737373662, 3.5334747474747323, ...","([0.0, 1.7560808080807975, 3.512161616161595, ...","([0.023999999997613486, 1.790212121209767, 3.5...","([0.008000000001629815, 1.7742222222238497, 3....","([3.0750000000043656, 4.810676767681081, 6.546...","[9.078805598323704, 8.944030976118826, 8.86207...",...,-21.679017,-10.431949,5.246347,2.671385,9.235699,-6.026532,-10.133274,Ia,True,1.11208
SN826467,"[[1.9500000000043656, 4.538, 6.571], [6.930000...","[[0.9650000000037835, 2.941, 5.424], [6.953000...","[[7.953000000001339, 1.836, 2.66], [8.94200000...","[[0.0, 4.296, 4.134], [2.993000000002212, 0.52...","[0.0, 1.2423131313131455, 2.484626262626291, 3...","([1.9500000000043656, 3.141878787883145, 4.333...","([0.9650000000037835, 2.1972828282866033, 3.42...","([7.953000000001339, 9.104444444445805, 10.255...","([0.0, 1.2423131313131455, 2.484626262626291, ...","[9.49690063066019, 10.62880940150936, 11.54852...",...,77.607987,-55.888939,15.411336,-25.509388,14.424325,-9.224315,47.200197,Ia,True,1.11368


### Auto ML 50 models 

In [12]:
start_time = time.time()

aml = auto_ML(df_smaller_half, 50, .8)

print("--- %s seconds ---" % (time.time() - start_time))

Checking whether there is an H2O instance running at http://localhost:54323 . connected.


H2O_cluster_uptime:,41 mins 01 secs
H2O_cluster_timezone:,Europe/Paris
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.6
H2O_cluster_version_age:,11 months and 26 days !!!
H2O_cluster_name:,H2O_from_python_fm_183dwg
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.805 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%
Leaderboard:  

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_BestOfFamily_AutoML_20220815_153547,0.00432703,0.0657801,0.00432703,0.0501768,0.0431453
StackedEnsemble_AllModels_AutoML_20220815_153547,0.00433798,0.0658633,0.00433798,0.0509869,0.0429475
GBM_4_AutoML_20220815_153547,0.00439977,0.0663307,0.00439977,0.0507714,0.043523
GBM_1_AutoML_20220815_153547,0.00449461,0.0670419,0.00449461,0.0514801,0.0440819
GBM_grid__1_AutoML_20220815_153547_model_2,0.00453357,0.0673318,0.00453357,0.0514682,0.0442941
DRF_1_AutoML_20220815_153547,0.00460153,0.0678346,0.00460153,0.0529875,0.0447334
GBM_2_AutoML_20220815_153547,0.00462148,0.0679815,0.00462148,0.0523854,0.0446793
GBM_3_AutoML_20220815_153547,0.00464593,0.068161,0.00464593,0.0520225,0.0447913
XGBoost_grid__1_AutoML_20220815_153547_model_3,0.00467918,0.0684045,0.00467918,0.0525889,0.0451596
XRT_1_AutoML_20220815_153547,0.0046874,0.0684646,0.0046874,0.0532152,0.0451357


 

Leader:  Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_AutoML_20220815_153547

No model summary for this model

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 0.0008357955824759544
RMSE: 0.028910129409533164
MAE: 0.02162485639419097
RMSLE: 0.01860894207650099
R^2: 0.9642185251427361
Mean Residual Deviance: 0.0008357955824759544
Null degrees of freedom: 2035
Residual degrees of freedom: 2029
Null deviance: 47.55756470937104
Residual deviance: 1.7016798059210432
AIC: -8635.47183324821

ModelMetricsRegressionGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 0.004249864232098549
RMSE: 0.06519098275144
MAE: 0.050410986172740035
RMSLE: 0.04231216159714914
R^2: 0.8180578919582939
Mean Residual Deviance: 0.004249864232098549
Null degrees of freedom: 2035
Residual degrees of freedom: 2030
Null deviance: 47.575823279283526
Residual deviance: 8.652723576552646
AIC: -5326.410033389886
 

-

In [13]:
save_aml_models(aml)

Model StackedEnsemble_BestOfFamily_AutoML_20220815_153547 saved.
Model StackedEnsemble_AllModels_AutoML_20220815_153547 saved.
Model GBM_4_AutoML_20220815_153547 saved.
Model GBM_1_AutoML_20220815_153547 saved.
Model GBM_grid__1_AutoML_20220815_153547_model_2 saved.
Model DRF_1_AutoML_20220815_153547 saved.
Model GBM_2_AutoML_20220815_153547 saved.
Model GBM_3_AutoML_20220815_153547 saved.
Model XGBoost_grid__1_AutoML_20220815_153547_model_3 saved.
Model XRT_1_AutoML_20220815_153547 saved.
Model XGBoost_3_AutoML_20220815_153547 saved.
Model XGBoost_grid__1_AutoML_20220815_153547_model_9 saved.
Model XGBoost_grid__1_AutoML_20220815_153547_model_11 saved.
Model GBM_grid__1_AutoML_20220815_153547_model_5 saved.
Model XGBoost_grid__1_AutoML_20220815_153547_model_8 saved.
Model XGBoost_grid__1_AutoML_20220815_153547_model_2 saved.
Model XGBoost_grid__1_AutoML_20220815_153547_model_13 saved.
Model GBM_grid__1_AutoML_20220815_153547_model_7 saved.
Model XGBoost_grid__1_AutoML_20220815_153547_

### Making predictions.

In [16]:
best_model = h2o.import_mojo('../../models/mojo_50_ensemble_08validation_lowerzhalf/StackedEnsemble_AllModels_AutoML_20220815_153547.zip')
preds, len_train, len_validation = gen_predictions(df_higher_half, best_model, validation_ratio=0.999999)

generic Model Build progress: |███████████████████████████████████████████| 100%
Model Details
H2OGenericEstimator :  Import MOJO Model
Model Key:  Generic_model_python_1660568080817_4

No model summary for this model

ModelMetricsRegressionGeneric: generic
** Reported on train data. **

MSE: 0.0010809482859085824
RMSE: 0.032877777995305314
MAE: 0.025270562878328837
RMSLE: 0.020716852249223643
Mean Residual Deviance: 0.0010809482859085824

ModelMetricsRegressionGeneric: generic
** Reported on cross-validation data. **

MSE: 0.004163390464107778
RMSE: 0.06452434009044787
MAE: 0.04997816615464649
RMSLE: 0.041827574841858894
Mean Residual Deviance: 0.004163390464107778

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
generic prediction progress: |████████████████████████████████████████████| 100%

 2539 train/test objects  
 and 0 validation objects


In [17]:
preds

predict
0.679454
0.573153
0.679984
0.62403
0.574205
0.675677
0.68061
0.689046
0.68455
0.674355


### Manual RMSE verification

In [20]:
pd_preds = preds.as_data_frame()['predict'].tolist()
true_y = list(df_higher_half.iloc[-len_validation:].loc[:,'REDSHIFT_SPEC'])
RMSE = (mean_squared_error(list(df_higher_half.iloc[-len_validation:].loc[:,'REDSHIFT_SPEC']), pd_preds))**(1/2)
RMSE

0.2622788086809962